<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [ ]:
## Task 1: Mark all launch sites on a map

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [4]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [5]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [13]:
# Create lists of coordinates, names and colors
launch_site_map_coordinates = launch_sites_df[['Lat', 'Long']].values.tolist()
launch_site_map_location_names = list(launch_sites_df['Launch Site'])
launch_site_map_circle_colors = ['#ff8800', '#007799', '#55dd00', '#aa00aa']

In [14]:
launch_site_map_circle_colors

['#ff8800', '#007799', '#55dd00', '#aa00aa']

In [27]:
launch_site_map_coordinates

[[28.56230197, -80.57735648],
 [28.56319718, -80.57682003],
 [28.57325457, -80.64689529],
 [34.63283416, -120.6107455]]

In [21]:
# Start location is NASA Johnson Space Center
nasa_coordinates = [29.559684888503615, -95.0830971930759]
launch_site_map = folium.Map(location=nasa_coordinate, zoom_start=10)
launch_site_map

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [22]:
# Create a blue circle at NASA Johnson Space Center's coordinate
circle = folium.Circle(nasa_coordinates, radius=1000, color='#0055dd', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
launch_site_map.add_child(circle)

# MARKER - Create an icon showing an abbreviation of the name 'NASA Johnson Space Center'
marker = folium.map.Marker(nasa_coordinates, icon=DivIcon(icon_size=(10,10),
                                                         icon_anchor=(-20,0),
                                                         html='<div style="font-size: 16px; color:#000;"><b>%s</b></div>' % 'NASA_JSC'
                                                        )
    )
launch_site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [30]:
import folium
from folium.features import DivIcon

# Define coordinates for NASA Johnson Space Center
nasa_coordinates = [29.559684888503615, -95.0830971930759]

# Initialize map with a better zoom level for all sites to be visible
launch_site_map = folium.Map(location=nasa_coordinates, zoom_start=6)

# Create lists of coordinates, names, and colors
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_site_map_coordinates = launch_sites_df[['Lat', 'Long']].values.tolist()
launch_site_map_location_names = list(launch_sites_df['Launch Site'])
launch_site_map_circle_colors = ['#ff8800', '#007799', '#55dd00', '#aa00aa']

# Add NASA marker separately
folium.Marker(
    location=nasa_coordinates,
    popup='NASA Johnson Space Center',
    icon=DivIcon(
        icon_size=(200, 36),
        icon_anchor=(100, 0),
        html='<div style="font-size:14px; color:blue; text-align:center;">NASA Johnson Space Center</div>'
    )
).add_to(launch_site_map)

# Add circles and markers to the map
for index, coordinates in enumerate(launch_site_map_coordinates):
    # Set variables
    circle_coordinates = coordinates
    circle_popup_name = launch_site_map_location_names[index]
    circle_color = launch_site_map_circle_colors[index]

    # Add Circle
    folium.Circle(
        location=circle_coordinates,
        radius=300,  # Adjusted radius for better visibility
        color=circle_color,
        fill=True,
        fill_opacity=0.6
    ).add_child(folium.Popup(circle_popup_name)).add_to(launch_site_map)

    # Add Marker with adjusted font size and placement
    folium.Marker(
        location=[coordinates[0], coordinates[1] + 0.01],  # Slightly offset to prevent overlap
        icon=DivIcon(
            icon_size=(200, 36),
            icon_anchor=(100, 0),
            html=f'<div style="font-size:12px; color:black; text-align:center;">{circle_popup_name}</div>'
        )
    ).add_to(launch_site_map)

# Display the map
launch_site_map



In [36]:
import folium
from folium.features import DivIcon

# Define coordinates for NASA Johnson Space Center
nasa_coordinates = [29.559684888503615, -95.0830971930759]

# Initialize map with a better zoom level for all sites to be visible
site_map = folium.Map(location=nasa_coordinates, zoom_start=5)

# Create lists of coordinates, names, and colors
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

# Define unique colors for each launch site
launch_site_colors = ['#FF5733', '#33FF57', '#3357FF', '#FF33A1']

# Function to add colored circles and labels
def create_point_coordinates(row, color):
    coordinate = [row['Lat'], row['Long']]
    circle = folium.Circle(
        location=coordinate,
        radius=150,
        color=color,  # Unique color for each site
        fill=True,
        fill_opacity=0.6
    ).add_child(folium.Popup(f'<b>{row["Launch Site"]}</b>', max_width=300))

    marker = folium.Marker(
        location=[coordinate[0] + 0.02, coordinate[1]],  # Slight offset to avoid overlap
        icon=DivIcon(
            icon_size=(150, 36),
            icon_anchor=(75, 0),
            html=f'<div style="font-size: 14px; color:{color}; text-align:center; font-weight:bold;">{row["Launch Site"]}</div>'
        )
    )

    site_map.add_child(circle)
    site_map.add_child(marker)

# Apply the function to each row with unique colors
for idx, row in launch_sites_df.iterrows():
    create_point_coordinates(row, launch_site_colors[idx % len(launch_site_colors)])

# Add NASA marker separately
folium.Marker(
    location=nasa_coordinates,
    popup='NASA Johnson Space Center',
    icon=DivIcon(
        icon_size=(250, 50),
        icon_anchor=(125, 0),
        html='<div style="font-size:16px; color:#0000FF; text-align:center; font-weight:bold;">NASA Johnson Space Center</div>'
    )
).add_to(site_map)

# Display the map
site_map




The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [ ]:
Task 2: Mark the success/failed launches for each site on the map

In [37]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [30]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [38]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

spacex_df['marker_color'] = list(map(lambda x: 'green' if x==1 else 'red', spacex_df['class']))
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [46]:
import folium
from folium.features import DivIcon
from folium import plugins
# Define coordinates for NASA Johnson Space Center
nasa_coordinates = [29.559684888503615, -95.0830971930759]
# Initialize map with a better zoom level for all sites to be visible
site_map = folium.Map(location=nasa_coordinates, zoom_start=5)
# Create a marker cluster to hold all launch markers
marker_cluster = plugins.MarkerCluster().add_to(site_map)
# Add NASA marker separately
folium.Marker(
    location=nasa_coordinates,
    popup='NASA Johnson Space Center',
    icon=DivIcon(
        icon_size=(250, 50),
        icon_anchor=(125, 0),
        html='<div style="font-size:16px; color:#0000FF; text-align:center; font-weight:bold;">NASA Johnson Space Center</div>'
    )
).add_to(site_map)
# Function to add colored circles and labels for each launch site
def create_site_marker(row, color):
    coordinate = [row['Lat'], row['Long']]
    circle = folium.Circle(
        location=coordinate,
        radius=150,
        color=color,
        fill=True,
        fill_opacity=0.6
    ).add_child(folium.Popup(f'<b>{row["Launch Site"]}</b>', max_width=300))
    marker = folium.Marker(
        location=coordinate,
        icon=DivIcon(
            icon_size=(150, 36),
            icon_anchor=(75, 0),
            html=f'<div style="font-size: 14px; color:{color}; text-align:center; font-weight:bold;">{row["Launch Site"]}</div>'
        )
    )
    site_map.add_child(circle)
    site_map.add_child(marker)
# Grouping the launch sites and adding them to the map
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

launch_site_colors = ['#FF5733', '#33FF57', '#3357FF', '#FF33A1']  # Unique colors for each site
for idx, row in launch_sites_df.iterrows():
    create_site_marker(row, launch_site_colors[idx % len(launch_site_colors)])
# Add launch markers (success/failure) to the map
def add_launch_marker(row):
    lat, lng = row['Lat'], row['Long']
    marker_color = 'green' if row['class'] == 1 else 'red'
    marker = folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color='white', icon_color=marker_color),
        popup=f"{row['Launch Site']} - {'Success' if row['class'] == 1 else 'Failure'}"
    )
    marker_cluster.add_child(marker)
# Apply function to add markers for all launches
spacex_df.apply(add_launch_marker, axis=1)
# Display the map
site_map

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [47]:
# TASK 3: Calculate the distances between a launch site to its proximities

import folium
from folium.features import DivIcon
from math import sin, cos, sqrt, atan2, radians

# Distance Calculation Function
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

   
# CCAFS LC-40 Launch Site Coordinates
#launch_site_lat = 28.562302
#launch_site_lon = -80.577356

# Coordinates for Launch Site and Coastline
lat_lon_launch_site = [28.562302, -80.577356]
lat_lon_coastline = [28.56321, -80.56801]

# Extract Latitude and Longitude
launch_site_lat = lat_lon_launch_site[0]
launch_site_lon = lat_lon_launch_site[1]
coastline_lat = lat_lon_coastline[0]
coastline_lon = lat_lon_coastline[1]

# Calculate Distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print(f"Distance to coastline: {distance_coastline:.2f} KM")


# Folium Map Setup
site_map = folium.Map(location=lat_lon_launch_site, zoom_start=15)

# Add Distance Marker at the midpoint between launch site and coastline
midpoint = [(launch_site_lat + coastline_lat) / 2, (launch_site_lon + coastline_lon) / 2]
distance_marker = folium.Marker(
    midpoint,
    icon=DivIcon(
        icon_size=(100, 30),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_coastline)
    )
)
site_map.add_child(distance_marker)

# Add Polyline between Launch Site and Coastline
coordinates = [lat_lon_launch_site, lat_lon_coastline]
lines = folium.PolyLine(
    locations=coordinates,
    weight=2,
    color='blue',
    dash_array='5,5'
)
site_map.add_child(lines)

# Add Markers for Launch Site and Coastline
folium.Marker(
    lat_lon_launch_site,
    popup='Launch Site',
    icon=folium.Icon(color='green', icon='rocket')
).add_to(site_map)

folium.Marker(
    lat_lon_coastline,
    popup='Coastline',
    icon=folium.Icon(color='blue', icon='info-sign')
).add_to(site_map)

# Save Map
site_map.save('launch_site_to_coastline.html')

# Display the map (Jupyter Notebook only)
site_map


Distance to coastline: 0.92 KM


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [52]:
# CCAFS LC-40 Launch Site Coordinates
launch_site_lat = 28.562302
launch_site_lon = -80.577356
# Coordinates of the closet COASTLINE
coastline_lat = 28.56321
coastline_lon = -80.56801
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print(f'Distance to Coastline: {round(distance_coastline,2)} km')

# Create and add a folium.Marker on the selected point on the map
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(100,-10),
        html=f'<div style="font-size:12px; color:#222; font-weight:bold; width:200px;">Coastline {round(distance_coastline,2)} km</div>',
        )
    ).add_to(launch_site_map)

Distance to Coastline: 0.92 km


In [50]:
# CCAFS LC-40 Launch Site Coordinates
launch_site_lat = 28.562302
launch_site_lon = -80.577356
# Coordinates of the closet RAIL LINE
rail_line_lat = 28.57209
rail_line_lon = -80.58520

distance_rail_line = calculate_distance(launch_site_lat, launch_site_lon, rail_line_lat, rail_line_lon)
print(f'Distance to Rail Line: {round(distance_rail_line,2)} km')

# Create and add a folium.Marker on the selected point on the map
distance_marker = folium.Marker(
    [rail_line_lat, rail_line_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(-40,-20),
        html=f'<div style="font-size:12px; color:#222; font-weight:bold; width:200px;">Rail Line {round(distance_rail_line,2)} km</div>',
        )
    ).add_to(launch_site_map)

Distance to Rail Line: 1.33 km


In [51]:
# Coordinates of a point on the ROAD circling the launch site
road_lat = 28.561695
road_lon = -80.575553
distance_road = calculate_distance(launch_site_lat, launch_site_lon, road_lat, road_lon)
print(f'Distance to Road: {round(distance_road,2)} km')

# Create and add a folium.Marker on the selected point on the map
distance_marker = folium.Marker(
    [road_lat, road_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(10,0),
        html=f'<div style="font-size:12px; color:#222; font-weight:bold; width:160px;">Perimeter Road {round(distance_road,2)} km</div>',
        )
    ).add_to(launch_site_map)

Distance to Road: 0.19 km


In [54]:
# Coordinates of a City(Cape Canaveral, Florida) near to the launch site

city_lat = 28.6115
city_lon = -80.8077
distance_road = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)
print(f'Distance to Road: {round(distance_road,2)} km')

# Create and add a folium.Marker on the selected point on the map
distance_marker = folium.Marker(
    [road_lat, road_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(10,0),
        html=f'<div style="font-size:12px; color:#222; font-weight:bold; width:160px;">Perimeter Road {round(distance_road,2)} km</div>',
        )
    ).add_to(launch_site_map)

Distance to Road: 23.15 km


In [60]:
import folium.plugins as plugins
import folium
from folium.features import DivIcon
import math

# Haversine function to calculate the distance between two lat/lon points
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in km
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c  # Returns the distance in kilometers

# Initialize map with a center location (e.g., CCAFS LC-40 Launch Site)
launch_site_lat = 28.562302
launch_site_lon = -80.577356
launch_site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=12)

# Define feature coordinates
coastline_lat = 28.56321
coastline_lon = -80.56801
rail_line_lat = 28.57209
rail_line_lon = -80.58520
road_lat = 28.561695
road_lon = -80.575553
city_lat = 28.6115
city_lon = -80.8077

# Create and add markers for distances (Coastline, Rail Line, Road, City)
# Coastline
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(100, -10),
        html=f'<div style="font-size:12px; color:#222; font-weight:bold; width:200px;">Coastline {round(distance_coastline,2)} km</div>'
    )
).add_to(launch_site_map)

# Rail Line
distance_rail_line = calculate_distance(launch_site_lat, launch_site_lon, rail_line_lat, rail_line_lon)
folium.Marker(
    [rail_line_lat, rail_line_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(-40, -20),
        html=f'<div style="font-size:12px; color:#222; font-weight:bold; width:200px;">Rail Line {round(distance_rail_line, 2)} km</div>'
    )
).add_to(launch_site_map)

# Road
distance_road = calculate_distance(launch_site_lat, launch_site_lon, road_lat, road_lon)
folium.Marker(
    [road_lat, road_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(10, 0),
        html=f'<div style="font-size:12px; color:#222; font-weight:bold; width:160px;">Perimeter Road {round(distance_road, 2)} km</div>'
    )
).add_to(launch_site_map)

# City
distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)
folium.Marker(
    [city_lat, city_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(10, 0),
        html=f'<div style="font-size:12px; color:#222; font-weight:bold; width:160px;">Cape Canaveral {round(distance_city, 2)} km</div>'
    )
).add_to(launch_site_map)

# Create and add PolyLine paths between the launch site and the features
coastline_coordinates = [(launch_site_lat, launch_site_lon), (coastline_lat, coastline_lon)]
rail_line_coordinates = [(launch_site_lat, launch_site_lon), (rail_line_lat, rail_line_lon)]
road_coordinates = [(launch_site_lat, launch_site_lon), (road_lat, road_lon)]
city_coordinates = [(launch_site_lat, launch_site_lon), (city_lat, city_lon)]

# PolyLines to visualize paths
folium.PolyLine(locations=coastline_coordinates, weight=2, color='blue').add_to(launch_site_map)
folium.PolyLine(locations=rail_line_coordinates, weight=2, color='green').add_to(launch_site_map)
folium.PolyLine(locations=road_coordinates, weight=2, color='orange').add_to(launch_site_map)
folium.PolyLine(locations=city_coordinates, weight=2, color='purple').add_to(launch_site_map)

# Display the map
launch_site_map

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


In [ ]:
For CCAFS SLC-40:

Is CCAFS SLC-40 in close proximity to railways ? Yes​
Is CCAFS SLC-40 in close proximity to highways ? Yes​
Is CCAFS SLC-40 in close proximity to coastline ? Yes​
Do CCAFS SLC-40 keeps certain distance away from cities ? No

# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
